# Deploying PySyft on a Local Kubernetes Cluster

## Introduction
Welcome to our quick start guide for deploying PySyft on a local Kubernetes cluster! PySyft is a powerful framework for privacy-preserving machine learning, and deploying it on Kubernetes allows an easy way to quickly try out the full PySyft stack on your own system. This guide will walk you through the process step by step.

## Prerequisites
Before we begin, ensure you have the following prerequisites installed on your system:
1. [Docker](https://docs.docker.com/install/): Docker is required to create and manage containers.
2. [kubectl](https://kubernetes.io/docs/tasks/tools/#kubectl): kubectl is the command-line tool for interacting with Kubernetes clusters.
3. [k3d](https://k3d.io/v5.6.3/#installation): k3d is used to create local Kubernetes clusters.
4. [Helm](https://helm.sh/docs/intro/install/): Helm is the package manager for Kubernetes, used to install and manage applications on Kubernetes clusters.

## Deployment Steps

### 1. Create a Local Kubernetes Cluster
First, create a local Kubernetes cluster named "syft" using k3d:
```sh
k3d cluster create syft -p "8080:80@loadbalancer"
```

### 2. Add and Update Helm Repo for Syft
Add the Helm repository for PySyft and update it:
```sh
helm repo add openmined https://openmined.github.io/PySyft/helm
helm repo update openmined
```

### 3. Search for Available Syft Versions
Explore available versions of PySyft using Helm:
```sh
helm search repo openmined/syft --versions --devel
```

### 4. Set Your Preferred Syft Chart Version
Set the version of PySyft you want to install:
```sh
SYFT_VERSION="<paste the chart version number>"
```

### 5. Provision Helm Charts
Install PySyft on the Kubernetes cluster with your preferred version:
```sh
helm install my-syft openmined/syft --version $SYFT_VERSION --namespace syft --create-namespace --set ingress.className="traefik"
```

<div class="alert alert-block alert-info">
<b>💡 Tip 1:</b>

If you want to deploy your Kubernetes cluster in a resource-constrained environment, use the following flags to override the default configurations. Please note that you will need at least 1 CPU and 2 GB of RAM on Docker, and some tests may not work in such low-resource environments:

```sh
helm install my-syft openmined/syft --version $SYFT_VERSION --namespace syft --create-namespace --set ingress.className="traefik" --set server.resourcesPreset=null --set seaweedfs.resourcesPreset=null --set postgres.resourcesPreset=null --set registry.resourcesPreset=null --set proxy.resourcesPreset=null --set frontend.resourcesPreset=null
```

</div>

<div class="alert alert-block alert-info">
<b>💡 Tip 2:</b>

If you would like to set your own default password even for the production style deployment, use the following command:

```sh
helm install my-syft openmined/syft --version $SYFT_VERSION --namespace syft --create-namespace --set ingress.className="traefik" --set global.randomizedSecrets=false --set server.secret.defaultRootPassword="changethis" --set seaweedfs.secret.s3RootPassword="admin" --set postgres.secret.rootPassword="example"
```
</div>


## Working with the local Kubernetes deployment

PySyft makes it very simple to connect to your existing Syft cluster by providing the `sy.orchestra` interface. You can connect to the datasite by executing these steps in your jupyter notebook:

```python
# syft absolute
import syft as sy

server = sy.orchestra.launch(name="syft-example-datasite-1", deploy_to="remote")
```

This will return a server handle by connecting to `http://localhost:8080` which is the default host and port where your kubernetes cluster will be running. You can connect to a different host and port by setting the environment variables `SERVER_URL` and `SERVER_PORT`.

```python
import os

os.environ["SERVER_URL"] = "<http://CUSTOM-HOST>"
os.environ["SERVER_PORT"] = "<CUSTOM-PORT-NUMBER>"
```

Now, we are ready to start using the datasite. Since helm is a product grade deployement stack, the datasite comes with a randomized password for the default email credentials for the admin. Either run with Step 5 with your custom password or to extract the randomized password using `kubectl`, run the following command (in case you use a custom cluster name in step 1, replace `--context=k3d-$CLUSTER_NAME` appropriately): 

```sh
kubectl --context=k3d-syft get secret backend-secret -n syft -o jsonpath='{.data.defaultRootPassword}' | base64 --decode
```

and use the password instead of "changethis" below:

```python
client = server.login(email="info@openmined.org", password="changethis")
```

Once you are logged in, you are ready to start using the datasite, for instance for creating a dataset (this one is empty, just as a example).

```python3
dataset = sy.Dataset(name="my dataset", asset_list=[])
client.upload_dataset(dataset)
```

## Next Steps
Congratulations! You have successfully deployed PySyft on your local Kubernetes cluster. Now, you can explore its capabilities and use cases through our API example notebooks:

📝 [API Example Notebooks](../../api)
- [00-load-data.ipynb](../../api/0.8/00-load-data.ipynb)
- [01-submit-code.ipynb](../../api/0.8/01-submit-code.ipynb)
- [02-review-code-and-approve.ipynb](../../api/0.8/02-review-code-and-approve.ipynb)
- [03-data-scientist-download-result.ipynb](../../api/0.8/03-data-scientist-download-result.ipynb)
- [04-pytorch-example.ipynb](../../api/0.8/04-pytorch-example.ipynb)
- [05-custom-policy.ipynb](../../api/0.8/05-custom-policy.ipynb)
- [06-multiple-code-requests.ipynb](../../api/0.8/06-multiple-code-requests.ipynb)
- [07-datasite-register-control-flow.ipynb](../../api/0.8/07-datasite-register-control-flow.ipynb)
- [08-code-version.ipynb](../../api/0.8/08-code-version.ipynb)
- [09-blob-storage.ipynb](../../api/0.8/09-blob-storage.ipynb)
- [10-container-images.ipynb](../../api/0.8/10-container-images.ipynb)
- [11-container-images-k8s.ipynb](../../api/0.8/11-container-images-k8s.ipynb)

Feel free to explore these notebooks to get started with PySyft and unlock its full potential for privacy-preserving machine learning!